* Violência e desempenho escolar (Inferência Causal)
* Dados externos (IBGE, população, renda per capita)
    - Nome município, código IBGE, etc.
* Escala da cidade afeta o desempenho escolar
* Violência
    - Dados da Bahia (homícidio, roubo, furto)
* EDA
* Código da escola do Aluno
    * Censo da Educação Básica
* Limpeza da base
* Visões

Tipo da variável: categórica ou numérica


*GERAL*

*Visões geral*:

* Quantidade de inscritos por ano
* Percentual de ausentes nos dois dias
* Inscritos por tipo de língua
* Inscritos por escola
* Inscritos por gênero
* Composição da quantidade de inscritos no por ano
    * Proporção de gênero
    * Proporção de escolas
    * Proporção dependência administrativa da escola
    * ...
* Status da Redação
    
*Visão Série Temporal*

* Quantidade de inscritos na série histórica (2010-2021)
* Sexo
* Cor/Raça
* Estado Civil
* Conclusão Ensino Médio
* Tipo Escola
* Dependência administrativa 
* Média nota no tempo por variável
    * Sexo
    * Tipo escola
    * Dependência administrativa
    * Região geográfica
    * UF
    * Cor/raça
    * Renda
    * Escolaridade do pai/mãe
    * Tamanho da família

*Composição Geral*

Proporção e valor absoluto
* Sexo e localidade (UF/Região geográfica)
* Cor/Raça
* Tipo Escola
* Dependência administrativa 
    
*Visão Média Geral*

* Média global
* Média por gênero
* Média por etnia
* Média por tipo de escola
* Média por tempo de conclusão
* Média por faixa etária
* Média/Nota Máxima por regiões

*Visão abstenção/falta*

* Falta por gênero
* Falta por regiões geográficas
* Falta por UF
* Falta por município
* Falta por faixa etária
* Falta por disciplina
* Falta por etnia
* Falta por tipo de escola


*ESTATÍSTICA*
* Univariada
    * Filtros : Prova / Ano / Variável
    * Boxplot pela variável
    * Gráfico das notas densidade por cada grupo da variável
    * Gráfico de Barras
        * Quebrar por sexo
    * Situação de conclusão
    * Escolaridade do pai/mãe
    * Tamanho da família
    * Acesso à internet
    * Computador
* Análise multivariada

*RANKING*

    
* Quantidade de inscritos na Bahia nos últimos anos
* Composição da quantidade de inscritos na Bahia
* Mapa com a quantidade de inscritos (2019-2020) por UF
* Mapa com quantidade de inscritos por município (2019-2020)

* Ranking de Média nas provas por UF pré-pandemia (2019) e durante a pandemia (2020)
* Ranking de Média nas provas por município pré-pandemia e durante a pandemia
* 

* Visão UF
    * Quantidade de Inscritos
    * Falta
    * Proporções
* Visão Município
* Visão Escola

In [2]:
pip install geobr

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import geobr
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 250);pd.set_option('display.max_rows', 250)
import os


In [8]:
def create_columns_region(uf):
    norte        = ['AM', 'RR', 'AP', 'PA', 'TO', 'RO', 'AC']
    nordeste     = ['MA', 'PI', 'CE', 'RN', 'PE', 'PB', 'SE', 'AL', 'BA']
    centro_oeste = ['MT', 'MS', 'GO']
    sudeste      = ['SP', 'RJ', 'ES', 'MG']
    sul          = ['PR', 'RS', 'SC']
    df           = ['DF']
    
    if uf in norte :
        return 'Norte'
    elif uf in nordeste :
        return 'Nordeste'
    elif uf in centro_oeste : 
        return 'Centro-Oeste'
    elif uf in sudeste : 
        return 'Sudeste'
    elif uf in sul : 
        return 'Sul'
    elif uf in df :
        return 'DF'
    else:
        return 'missing'
    
    
    
def build_statistics_metrics(ano_enem : int, 
                             lista_provas : list, 
                             df_enem : pd.DataFrame, 
                             cols_to_group : list, 
                             list_with_metrics = None):
    if list_with_metrics:
        list_metrics = list_with_metrics
    else:
        list_metrics = { 'MIN_': np.min, 'AVG_' :np.mean, 'MED_' : np.median, 'MAX_' : np.max, 'STD_' : np.std}
    
    def get_statistics(ano_enem, lista_provas, metric, df_enem, cols_to_group, metric_label):

        agg_result = (df_enem
         .query(f'NU_ANO == {ano_enem}')
         .groupby(cols_to_group, as_index = False)
         .agg({prova:metric for prova in lista_provas})
         .rename(columns = {prova : metric_label+'_'.join(prova.split('_')[1:]) for prova in lista_provas})
        )

        return agg_result
    
    result = [
        get_statistics(ano_enem, lista_provas, list_metrics[metric], df_enem, cols_to_group, metric).set_index(cols_to_group) 
      for metric in list_metrics
     ]

    result_df = (pd.concat(result, axis=1)
         .reset_index()
        )
    

    return result_df


def build_distribution_freq(df_enem, cols_to_group, cat_columns):
    
    def get_distribution_freq(df_enem, cols_to_group, cat_column):
        agg_result = (pd.get_dummies(df_enem[cols_to_group+[cat_column]],
               columns = [cat_column])
                 .groupby(cols_to_group).sum()
                )
        return agg_result
    
    result    = [get_distribution_freq(df_enem, cols_to_group, cat_column) for cat_column in cat_columns]
    result_df = (pd.concat(result, axis = 1))
    return result_df

def get_count(ano_enem, df_enem, cols_to_group, column, column_name = None):
    agg_result = (df_enem
         .query(f'NU_ANO == {ano_enem}')
         .groupby(cols_to_group)
         .agg({column:'count'})
         .rename(columns = {column : column_name} )
        )
    return agg_result

In [3]:
pwd

'D:\\Trabalho\\Microdados\\ENEM'

In [2]:
os.chdir('D:\Trabalho\Microdados\ENEM')

In [10]:
os.chdir("C:\\Users\\brunobezerra\\Downloads\\Livros e papers\\Microdados ENEM\\Microdados")

In [13]:
ano_enem = 2018
chunk_size = 1000
df_2018  = pd.read_csv(f'MICRODADOS_ENEM_{ano_enem}.csv', encoding = 'latin1', sep = ';', chunksize = chunk_size)
df_2018 = next(df_2018)
df_2018.shape

(1000, 78)

In [34]:
df_2018.to_excel('D:\\Repositório GIT\\Projetos\\Analise-de-Dados-ENEM\\Datasets\\Teste\\2018.xlsx', index=False)

In [14]:
ano_enem = 2019
chunk_size = 1000
df_2019  = pd.read_csv(f'MICRODADOS_ENEM_{ano_enem}.csv', encoding = 'latin1', sep = ';', chunksize = chunk_size)
df_2019 = next(df_2019)
df_2019.shape

(1000, 76)

In [36]:
df_2019.to_excel('D:\\Repositório GIT\\Projetos\\Analise-de-Dados-ENEM\\Datasets\\Teste\\2019.xlsx', index=False)

In [ ]:
df_2018.dtypes

In [ ]:
df_2019.dtypes

#### Sem limites

In [10]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

#### Normal

In [16]:
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

In [5]:
conn = sqlite3.connect('tb_enem.db')

#### União de dois df para testar

In [32]:
df_enem = pd.concat([df_2018, df_2019], ignore_index=True)
df_enem.shape

(2000, 78)

In [18]:
df_enem.head()

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,IN_TREINEIRO,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,CO_MUNICIPIO_PROVA,NO_MUNICIPIO_PROVA,CO_UF_PROVA,SG_UF_PROVA,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,CO_PROVA_CN,CO_PROVA_CH,CO_PROVA_LC,CO_PROVA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,TX_RESPOSTAS_CN,TX_RESPOSTAS_CH,TX_RESPOSTAS_LC,TX_RESPOSTAS_MT,TP_LINGUA,TX_GABARITO_CN,TX_GABARITO_CH,TX_GABARITO_LC,TX_GABARITO_MT,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,Q026,Q027
0,180007186625,2018,2,F,0.0,4,1,2,0,4,1.0,0,1500800.0,Ananindeua,15.0,PA,4.0,1.0,1.0,1500800,Ananindeua,15,PA,1,1,1,1,449.0,454.0,457.0,462.0,534.2,611.4,517.9,496.3,BABDABEECDBDCBCAAAEDCEDDEADADEACABEEBABCABADC,CBAEACBBADABADEDDCDABBEAEEBBDDCECBECDABADDEEA,99999BDAECEBACECBBBBEBBDABDAAAADECCCDADBCCBBAA...,CCDAECBEBDAACDACABADECBDAADACBEDDAACBEABABDCD,1,EBBEBACEDDEDDCBCCACBDDDAECAADBCEABEAEABDBCACE,CDECAABBDDECBEEDBEBBEBEDEEEBCEAEECECCDBDDCACD,CBEDBBBDCBECADBBBECBEAECABDADACDBABCECBBEDADAB...,ADCACEDDABEBBCBCADDACCCDEBBDAEEDBEXBCAEDABADA,1.0,120.0,40.0,40.0,120.0,40.0,360.0,E,E,D,D,2,F,A,C,D,A,A,A,B,A,A,B,A,B,C,B,B,D,B,C,B,B,D
1,180007186627,2018,3,F,0.0,4,1,1,1,1,1.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1503200,Igarapé-Açu,15,PA,1,1,1,1,447.0,454.0,457.0,459.0,506.9,477.8,495.3,440.6,BADEBBDBDECBADCCDCEDAEDBBEDCABEAECABCAEEAEACE,BEDBBDBCCABDADABAEDDBAABCDACBECCEACABAEAAAEAB,CDEBB99999ABADBBBEBDEBBADCDBCBDDCECDDAECAACDAA...,AEECACDEECDBBEEEABBEADDAADCEAEAEDCEBCDABDADEA,0,BDDEDBCACEBCCACDCDDAECAADBDBCEEAEAABEBEBBACED,CDECAABBDDECBEEDBEBBEBEDEEEBCEAEECECCDBDDCACD,CBEDBBBDCBECADBBBECBEAECABDADACDBABCECBBEDADAB...,ACEADCEBBDADAAEBBDDEDABADBEXCCCDBCAEEACBCDDAB,1.0,140.0,40.0,40.0,160.0,40.0,420.0,D,E,A,A,4,B,A,B,B,A,A,B,A,A,A,A,A,A,B,A,A,B,A,A,A,A,A
2,180007186628,2018,3,M,0.0,0,1,2,0,2,1.0,0,3155603.0,Rio Pardo de Minas,31.0,MG,2.0,1.0,1.0,3155603,Rio Pardo de Minas,31,MG,1,1,1,1,449.0,453.0,458.0,462.0,470.6,473.8,534.4,410.4,CEEEEDBDACBBDADABCCCBDBBEBCEDEAACBACEEEBEBBEE,ECBDABEADCECAEAEECBCBDEABCDEADCBBCDAECDDBDABC,ECBDB99999CBDDCEABEABDBDEEECCCDDCCCDBDBBAAECBC...,BCAAACEBDCDBACADEABACCBDADADBDBCDCDABBDCBCDCB,0,EBBEBACEDDEDDCBCCACBDDDAECAADBCEABEAEABDBCACE,ECCBBEBEDBEDBDDAEECDECBECACDEDEECDECAABBDEBCE,BCDEBDBCBBACDCBEAECBABADBBECECBDBBEABDADACEDEC...,ADCACEDDABEBBCBCADDACCCDEBBDAEEDBEXBCAEDABADA,1.0,120.0,120.0,100.0,120.0,140.0,600.0,B,B,A,A,7,B,A,B,D,B,B,B,A,A,A,A,A,A,B,B,A,D,A,A,A,B,A
3,180007186629,2018,2,F,0.0,3,1,2,0,4,1.0,0,1500800.0,Ananindeua,15.0,PA,4.0,1.0,1.0,1500800,Ananindeua,15,PA,1,1,1,1,450.0,452.0,456.0,461.0,588.9,663.3,614.5,711.5,AAEBECBAEAECEBAABADCDBDCDBAEDACBCDCDDCDEEBBEC,ACADBCEBDAACCDAEBEEDBDDDBEECDACAABDAABABEBECC,99999CBBEBDBCEDDCAAEDAEAEABEDCABBBCCDCDDDDCACE...,DABDACDCD*ACCDDBBDADEABBEEADAECABADBDDA*CDAEA,1,ACEDEAEAEBECAADABBDBCEDEDDCDDABCCACBCACEBDEBB,AEECDECBEBDCACDEDEEDBDDEBCECDECAABEDBEBBEBECC,BBEDCCBBDBBBEADECACEDBEADABEDBABAECCBEABDDBACD...,EBBDEECCCDACEADCBCAAEBBCBCADAEDABADDADDABDBEX,1.0,160.0,160.0,180.0,180.0,120.0,800.0,F,F,D,D,4,I,A,B,D,A,A,B,A,B,A,A,A,A,C,A,B,D,B,A,B,B,B
4,180007186631,2018,11,F,0.0,5,1,1,6,1,1.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2610905,Pesqueira,26,PE,1,1,1,1,448.0,452.0,456.0,460.0,477.8,621.5,519.4,543.1,ADEEEEEBDCBBBBCAACCCEEEDDACCCDBDAACCACBACECDC,AACCCEEADEEACEBBBEECCCBEBCECDDDAEAEDEEABEDECB,99999CDEDDDDBADDCBEBBBBBBDCDCBBDEEDADEDBDDDEBC...,ACCABCCBBDAEDDCBEEEEEDECECABBDCECCECDDABEADDA,1,BCACEEBBBDACEDECAADDDABDBCCACDCBCEDEDABEBEAEA,AEECDECBEBDCACDEDEEDBDDEBCECDECAABEDBEBBEBECC,BBEDCCBBDBBBEADECACEDBEADABEDBABAECCBEABDDBACD...,ADAEDABBCAADBEXEBBDEEADCCCCDACEDDABCBCAAEBBDD,1.0,120.0,120.0,120.0,100.0,40.0,500.0,A,E,A,A,3,C,A,B,C,A,B,B,A,

In [34]:
df_enem.shape

(1931, 77)

In [38]:
df_enem.to_excel('D:\\Repositório GIT\\Projetos\\Analise-de-Dados-ENEM\\Datasets\\Teste\\enem.xlsx', index=False)

#### Original

#### Só para entender o código abaixo, a primeira parte ele cria uma lista "notas", com Notas de todas provas, depois ele inverte o booleano com "~" que eu não entendi. Também não entendi a criação da lista "lista_provas" sendo idêntica a lista "notas"


#### Segunda parte ele tira os treineiros e as colunas de respostas que não servem

#### Terceira parte usa a função de criação de região para agrupar

#### Quarto - Cálcula as abstenções 

#### Quinto - cria uma dummy para as escolas com numeração faltante da coluna "NO_MUNICIPIO_ESC"

#### Sexto - Faz as médias com a lista "notas" sem e com redação

In [33]:
# DATA PREP
notas = df_enem.columns[(df_enem.columns.str.contains('NU_NOTA_')) & (~df_enem.columns.str.contains('NU_NOTA_COMP'))].tolist()
lista_provas = df_enem.columns[(df_enem.columns.str.contains('NU_NOTA_')) & (~df_enem.columns.str.contains('NU_NOTA_COMP'))].tolist()

df_enem = (df_enem
           .query('IN_TREINEIRO != 1')
           .drop(columns = df_enem.columns[df_enem.columns.str.contains('TX_')].tolist()+['IN_TREINEIRO'])
          )


# REGIÃO GEOGRÁFICA
df_enem['TP_REGIAO_PROVA'] = df_enem.SG_UF_PROVA.apply(create_columns_region)
df_enem['TP_REGIAO_ESC'] = df_enem.SG_UF_PROVA.apply(create_columns_region)

# ABSTENÇÃO - PRIMEIRO DIA
df_enem.loc[(df_enem['TP_PRESENCA_CH'] == 0) & (df_enem['TP_PRESENCA_LC'] == 0), ['ABS_DIA_1']] = 1
df_enem.loc[(df_enem['TP_PRESENCA_CH'] != 0) & (df_enem['TP_PRESENCA_LC'] != 0), ['ABS_DIA_1']] = 0
# ABSTENÇÃO - SEGUNDO DIA
df_enem.loc[(df_enem['TP_PRESENCA_CN'] == 0) & (df_enem['TP_PRESENCA_MT'] == 0), ['ABS_DIA_2']] = 1
df_enem.loc[(df_enem['TP_PRESENCA_CN'] != 0) & (df_enem['TP_PRESENCA_MT'] != 0), ['ABS_DIA_2']] = 0
# ABSTENÇÃO FULL DAYS
df_enem.loc[(df_enem['ABS_DIA_1'] == 1) & (df_enem['ABS_DIA_2'] == 1), ['ABS_ALL']] = 1
df_enem.loc[(df_enem['ABS_DIA_1'] != 1) & (df_enem['ABS_DIA_2'] != 1), ['ABS_ALL']] = 0
# FLAG ESCOLA NULL
df_enem.loc[df_enem['NO_MUNICIPIO_ESC'].isna(), ['FLAG_NULL_ESC']]  = 1
df_enem.loc[df_enem['NO_MUNICIPIO_ESC'].notna(), ['FLAG_NULL_ESC']] = 0

# MÉDIAS
df_enem['MEDIA_OBJ'] = df_enem[[prova for prova in notas if prova != 'NU_NOTA_REDACAO']].mean(axis = 1)
df_enem['MEDIA_GLOBAL'] = df_enem[notas].mean(axis = 1)

In [23]:
df_enem.to_excel('D:\\Repositório GIT\\Projetos\\Analise-de-Dados-ENEM\\Datasets\\Teste\\enem1.xlsx', index=False)

In [7]:
df_enem.head()

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,TP_ENSINO,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,CO_UF_ESC,SG_UF_ESC,TP_DEPENDENCIA_ADM_ESC,TP_LOCALIZACAO_ESC,TP_SIT_FUNC_ESC,CO_MUNICIPIO_PROVA,NO_MUNICIPIO_PROVA,CO_UF_PROVA,SG_UF_PROVA,TP_PRESENCA_CN,TP_PRESENCA_CH,TP_PRESENCA_LC,TP_PRESENCA_MT,CO_PROVA_CN,CO_PROVA_CH,CO_PROVA_LC,CO_PROVA_MT,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,TP_LINGUA,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,Q001,Q002,Q003,Q004,Q005,Q006,Q007,Q008,Q009,Q010,Q011,Q012,Q013,Q014,Q015,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025,Q026,Q027,TP_REGIAO_PROVA,TP_REGIAO_ESC,ABS_DIA_1,ABS_DIA_2,ABS_ALL,FLAG_NULL_ESC,MEDIA_OBJ,MEDIA_GLOBAL
0,180007186625,2018,2,F,0.0,4,1,2,0,4,1.0,1500800.0,Ananindeua,15.0,PA,4.0,1.0,1.0,1500800,Ananindeua,15,PA,1.0,1.0,1.0,1.0,449.0,454.0,457.0,462.0,534.2,611.4,517.9,496.3,1,1.0,120.0,40.0,40.0,120.0,40.0,360.0,E,E,D,D,2,F,A,C,D,A,A,A,B,A,A,B,A,B,C,B,B,D,B,C,B,B,D,Norte,Norte,0.0,0.0,0.0,0.0,539.95,503.96
1,180007186627,2018,3,F,0.0,4,1,1,1,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1503200,Igarapé-Açu,15,PA,1.0,1.0,1.0,1.0,447.0,454.0,457.0,459.0,506.9,477.8,495.3,440.6,0,1.0,140.0,40.0,40.0,160.0,40.0,420.0,D,E,A,A,4,B,A,B,B,A,A,B,A,A,A,A,A,A,B,A,A,B,A,A,A,A,A,Norte,Norte,0.0,0.0,0.0,1.0,480.15,468.12
2,180007186628,2018,3,M,0.0,0,1,2,0,2,1.0,3155603.0,Rio Pardo de Minas,31.0,MG,2.0,1.0,1.0,3155603,Rio Pardo de Minas,31,MG,1.0,1.0,1.0,1.0,449.0,453.0,458.0,462.0,470.6,473.8,534.4,410.4,0,1.0,120.0,120.0,100.0,120.0,140.0,600.0,B,B,A,A,7,B,A,B,D,B,B,B,A,A,A,A,A,A,B,B,A,D,A,A,A,B,A,Sudeste,Sudeste,0.0,0.0,0.0,0.0,472.30,497.84
3,180007186629,2018,2,F,0.0,3,1,2,0,4,1.0,1500800.0,Ananindeua,15.0,PA,4.0,1.0,1.0,1500800,Ananindeua,15,PA,1.0,1.0,1.0,1.0,450.0,452.0,456.0,461.0,588.9,663.3,614.5,711.5,1,1.0,160.0,160.0,180.0,180.0,120.0,800.0,F,F,D,D,4,I,A,B,D,A,A,B,A,B,A,A,A,A,C,A,B,D,B,A,B,B,B,Norte,Norte,0.0,0.0,0.0,0.0,644.55,675.64
4,180007186631,2018,11,F,0.0,5,1,1,6,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2610905,Pesqueira,26,PE,1.0,1.0,1.0,1.0,448.0,452.0,456.0,460.0,477.8,621.5,519.4,543.1,1,1.0,120.0,120.0,120.0,100.0,40.0,500.0,A,E,A,A,3,C,A,B,C,A,B,B,A,A,A,A,A,A,B,B,A,C,A,B,A,A,A,Nordeste,Nordeste,0.0,0.0,0.0,1.0,540.45,532.36


#### VARS

In [ ]:
#for ano in np.arange(2018, 2023):
#    path = f'raw_data/MICRODADOS_ENEM_{str(ano)}.csv'
#    df_enem = pd.read_csv(path, encoding = 'latin1', sep = ';', nrows = 10)
#    display(df_enem)

In [8]:
municipality_view = ['TP_REGIAO_PROVA','SG_UF_PROVA', 'CO_MUNICIPIO_PROVA', 'NO_MUNICIPIO_PROVA']
uf_view           = ['TP_REGIAO_PROVA', 'SG_UF_PROVA']
list_metrics      = { 'MIN_': np.min, 'AVG_' :np.mean, 'MED_' : np.median, 'MAX_' : np.max, 'STD_' : np.std}

# TP_ANO_CONCLUIU, TP_ESTADO_CIVIL

cat_columns       = [
    'TP_FAIXA_ETARIA', 'TP_SEXO',
    'TP_COR_RACA', 'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO',
    'TP_ESCOLA', 'TP_ENSINO',  'TP_DEPENDENCIA_ADM_ESC',
    'TP_LOCALIZACAO_ESC', 'TP_SIT_FUNC_ESC', 'TP_PRESENCA_CN',
    'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT', 
]

In [9]:
# municipality view
build_municipality_features = [get_count(ano_enem, df_enem, municipality_view, 'NU_INSCRICAO', 'QUANTIDADE_INSCRITOS'),
 build_distribution_freq(df_enem, municipality_view, cat_columns),
 build_statistics_metrics(ano_enem, lista_provas, df_enem, municipality_view, list_metrics)
]

df_result_municipality = pd.concat(build_municipality_features, axis = 1).reset_index()
df_result_municipality.insert(0, 'NU_ANO', ano_enem)


# uf view
build_uf_features = [get_count(ano_enem, df_enem, uf_view, 'NU_INSCRICAO', 'QUANTIDADE_INSCRITOS'),
 build_distribution_freq(df_enem, uf_view, cat_columns),
 build_statistics_metrics(ano_enem, lista_provas, df_enem, uf_view, list_metrics)
]

df_result_uf = pd.concat(build_uf_features, axis = 1).reset_index()
df_result_uf.insert(0, 'NU_ANO', ano_enem)

In [10]:
df_result_municipality

,NU_ANO,TP_REGIAO_PROVA,SG_UF_PROVA,CO_MUNICIPIO_PROVA,NO_MUNICIPIO_PROVA,QUANTIDADE_INSCRITOS,TP_FAIXA_ETARIA_1,TP_FAIXA_ETARIA_2,TP_FAIXA_ETARIA_3,TP_FAIXA_ETARIA_4,TP_FAIXA_ETARIA_5,TP_FAIXA_ETARIA_6,TP_FAIXA_ETARIA_7,TP_FAIXA_ETARIA_8,TP_FAIXA_ETARIA_9,TP_FAIXA_ETARIA_10,TP_FAIXA_ETARIA_11,TP_FAIXA_ETARIA_12,TP_FAIXA_ETARIA_13,TP_FAIXA_ETARIA_14,TP_FAIXA_ETARIA_15,TP_FAIXA_ETARIA_16,TP_FAIXA_ETARIA_17,TP_FAIXA_ETARIA_18,TP_FAIXA_ETARIA_19,TP_FAIXA_ETARIA_20,TP_SEXO_F,TP_SEXO_M,TP_COR_RACA_0,TP_COR_RACA_1,TP_COR_RACA_2,TP_COR_RACA_3,TP_COR_RACA_4,TP_COR_RACA_5,TP_NACIONALIDADE_0,TP_NACIONALIDADE_1,TP_NACIONALIDADE_2,TP_NACIONALIDADE_3,TP_NACIONALIDADE_4,TP_ST_CONCLUSAO_1,TP_ST_CONCLUSAO_2,TP_ST_CONCLUSAO_3,TP_ST_CONCLUSAO_4,TP_ESCOLA_1,TP_ESCOLA_2,TP_ESCOLA_3,TP_ESCOLA_4,TP_ENSINO_1.0,TP_ENSINO_2.0,TP_ENSINO_3.0,TP_DEPENDENCIA_ADM_ESC_1.0,TP_DEPENDENCIA_ADM_ESC_2.0,TP_DEPENDENCIA_ADM_ESC_3.0,TP_DEPENDENCIA_ADM_ESC_4.0,TP_LOCALIZACAO_ESC_1.0,TP_LOCALIZACAO_ESC_2.0,TP_SIT_FUNC_ESC_1.0,TP_SIT_FUNC_ESC_2.0,TP_SIT_FUNC_ESC_3.0,TP_SIT_FUNC_ESC_4.0,TP_PRESENCA_CN_0.0,TP_PRESENCA_CN_1.0,TP_PRESENCA_CN_2.0,TP_PRESENCA_CH_0.0,TP_PRESENCA_CH_1.0,TP_PRESENCA_CH_2.0,TP_PRESENCA_LC_0.0,TP_PRESENCA_LC_1.0,TP_PRESENCA_LC_2.0,TP_PRESENCA_MT_0.0,TP_PRESENCA_MT_1.0,TP_PRESENCA_MT_2.0,MIN_NOTA_CN,MIN_NOTA_CH,MIN_NOTA_LC,MIN_NOTA_MT,MIN_NOTA_REDACAO,AVG_NOTA_CN,AVG_NOTA_CH,AVG_NOTA_LC,AVG_NOTA_MT,AVG_NOTA_REDACAO,MED_NOTA_CN,MED_NOTA_CH,MED_NOTA_LC,MED_NOTA_MT,MED_NOTA_REDACAO,MAX_NOTA_CN,MAX_NOTA_CH,MAX_NOTA_LC,MAX_NOTA_MT,MAX_NOTA_REDACAO,STD_NOTA_CN,STD_NOTA_CH,STD_NOTA_LC,STD_NOTA_MT,STD_NOTA_REDACAO
0,2018,Centro-Oeste,GO,5200134,Acreúna,585,10.0,120.0,162.0,76.0,46.0,37.0,21.0,17.0,10.0,18.0,29.0,17.0,12.0,3.0,6.0,1.0,0.0,0.0,0.0,0.0,334.0,251.0,14.0,168.0,48.0,326.0,26.0,3.0,0.0,559.0,24.0,2.0,0.0,259.0,320.0,4.0,2.0,301.0,255.0,9.0,20.0,376.0,2.0,50.0,0.0,255.0,9.0,20.0,284.0,0.0,284.0,0.0,0.0,0.0,203.0,382.0,0.0,170.0,415.0,0.0,170.0,415.0,0.0,203.0,382.0,0.0,367.2,397.4,335.7,377.1,0.0,480.330628,543.096386,508.340241,513.198953,464.385542,472.05,549.7,509.60,498.75,440.0,690.2,717.6,713.9,830.3,980.0,68.492420,77.749789,69.691315,89.681400,183.521763
1,2018,Centro-Oeste,GO,5200258,Águas Lindas de Goiás,4613,98.0,585.0,947.0,625.0,442.0,307.0,210.0,178.0,165.0,115.0,419.0,244.0,148.0,71.0,37.0,16.0,2.0,3.0,1.0,0.0,2920.0,1693.0,138.0,854.0,717.0,2659.0,218.0,27.0,1.0,4466.0,139.0,2.0,5.0,2661.0,1822.0,65.0,65.0,3139.0,1310.0,6.0,158.0,2971.0,18.0,514.0,63.0,1310.0,6.0,158.0,1536.0,1.0,1534.0,2.0,1.0,0.0,1589.0,3024.0,0.0,1286.0,3326.0,1.0,1286.0,3326.0,1.0,1589.0,3024.0,0.0,362.8,0.0,0.0,0.0,0.0,468.132176,539.311756,501.029495,495.172983,447.059531,458.60,548.0,503.60,484.60,440.0,683.7,756.0,710.9,817.3,980.0,58.374785,80.763986,69.724554,77.175843,174.045748
2,2018,Centro-Oeste,GO,5200308,Alexânia,563,5.0,97.0,156.0,79.0,48.0,39.0,16.0,16.0,10.0,19.0,42.0,15.0,10.0,5.0,4.0,2.0,0.0,0.0,0.0,0.0,347.0,216.0,28.0,167.0,45.0,297.0,23.0,3.0,0.0,544.0,16.0,1.0,2.0,253.0,290.0,14.0,6.0,304.0,237.0,0.0,22.0,333.0,3.0,69.0,0.0,237.0,0.0,22.0,259.0,0.0,259.0,0.0,0.0,0.0,204.0,359.0,0.0,158.0,405.0,0.0,158.0,405.0,0.0,204.0,359.0,0.0,362.5,0.0,0.0,372.9,0.0,468.272981,542.224938,502.508889,508.850975,437.827160,458.60,554.2,502.50,496.80,420.0,753.7,706.8,688.0,755.7,960.0,61.020997,85.382283,68.391987,82.266818,184.812508
3,2018,Centro-Oeste,GO,5201108,Anápolis,11353,99.0,1608.0,2518.0,1443.0,947.0,773.0,556.0,490.0,358.0,298.0,1008.0,517.0,321.0,220.0,119.0,40.0,29.0,8.0,1.0,0.0,6609.0,4744.0,257.0,3546.0,1080.0,5879.0,542.0,49.0,1.0,10954.0,360.0,9.0,29.0,6695.0,4396.0,110.0,152.0,7519.0,3194.0,12.0,628.0,6592.0,36.0,1090.0,74.0,3194.0,12.0,628.0,3752.0,156.0,3908.0,0.0,0.0,0.0,3970.0,7383.0,0.0,3433.0,7916.0,4.0,3433.0,7916.0,4.0,3970.0,7383.0,0.0,0.0,0.0,0.0,0.0,0.0,502.945591,571.643709,529.078575,539.298524,530.449722,492.40,585.3,532.65,521.00,520.0,782.8,810.6,785.0,976.5,980.0,78.669321,80.452584,73.722398,105.137945,202.873483
4,201

In [11]:
df_result_uf

,NU_ANO,TP_REGIAO_PROVA,SG_UF_PROVA,QUANTIDADE_INSCRITOS,TP_FAIXA_ETARIA_1,TP_FAIXA_ETARIA_2,TP_FAIXA_ETARIA_3,TP_FAIXA_ETARIA_4,TP_FAIXA_ETARIA_5,TP_FAIXA_ETARIA_6,TP_FAIXA_ETARIA_7,TP_FAIXA_ETARIA_8,TP_FAIXA_ETARIA_9,TP_FAIXA_ETARIA_10,TP_FAIXA_ETARIA_11,TP_FAIXA_ETARIA_12,TP_FAIXA_ETARIA_13,TP_FAIXA_ETARIA_14,TP_FAIXA_ETARIA_15,TP_FAIXA_ETARIA_16,TP_FAIXA_ETARIA_17,TP_FAIXA_ETARIA_18,TP_FAIXA_ETARIA_19,TP_FAIXA_ETARIA_20,TP_SEXO_F,TP_SEXO_M,TP_COR_RACA_0,TP_COR_RACA_1,TP_COR_RACA_2,TP_COR_RACA_3,TP_COR_RACA_4,TP_COR_RACA_5,TP_NACIONALIDADE_0,TP_NACIONALIDADE_1,TP_NACIONALIDADE_2,TP_NACIONALIDADE_3,TP_NACIONALIDADE_4,TP_ST_CONCLUSAO_1,TP_ST_CONCLUSAO_2,TP_ST_CONCLUSAO_3,TP_ST_CONCLUSAO_4,TP_ESCOLA_1,TP_ESCOLA_2,TP_ESCOLA_3,TP_ESCOLA_4,TP_ENSINO_1.0,TP_ENSINO_2.0,TP_ENSINO_3.0,TP_DEPENDENCIA_ADM_ESC_1.0,TP_DEPENDENCIA_ADM_ESC_2.0,TP_DEPENDENCIA_ADM_ESC_3.0,TP_DEPENDENCIA_ADM_ESC_4.0,TP_LOCALIZACAO_ESC_1.0,TP_LOCALIZACAO_ESC_2.0,TP_SIT_FUNC_ESC_1.0,TP_SIT_FUNC_ESC_2.0,TP_SIT_FUNC_ESC_3.0,TP_SIT_FUNC_ESC_4.0,TP_PRESENCA_CN_0.0,TP_PRESENCA_CN_1.0,TP_PRESENCA_CN_2.0,TP_PRESENCA_CH_0.0,TP_PRESENCA_CH_1.0,TP_PRESENCA_CH_2.0,TP_PRESENCA_LC_0.0,TP_PRESENCA_LC_1.0,TP_PRESENCA_LC_2.0,TP_PRESENCA_MT_0.0,TP_PRESENCA_MT_1.0,TP_PRESENCA_MT_2.0,MIN_NOTA_CN,MIN_NOTA_CH,MIN_NOTA_LC,MIN_NOTA_MT,MIN_NOTA_REDACAO,AVG_NOTA_CN,AVG_NOTA_CH,AVG_NOTA_LC,AVG_NOTA_MT,AVG_NOTA_REDACAO,MED_NOTA_CN,MED_NOTA_CH,MED_NOTA_LC,MED_NOTA_MT,MED_NOTA_REDACAO,MAX_NOTA_CN,MAX_NOTA_CH,MAX_NOTA_LC,MAX_NOTA_MT,MAX_NOTA_REDACAO,STD_NOTA_CN,STD_NOTA_CH,STD_NOTA_LC,STD_NOTA_MT,STD_NOTA_REDACAO
0,2018,Centro-Oeste,GO,163407,2402.0,24546.0,35530.0,20566.0,13467.0,9964.0,7687.0,6260.0,5208.0,4260.0,14271.0,7883.0,5273.0,3067.0,1686.0,841.0,326.0,124.0,36.0,10.0,95208.0,68199.0,4758.0,47179.0,20370.0,83506.0,6783.0,811.0,42.0,158036.0,4840.0,139.0,350.0,94488.0,65013.0,1800.0,2106.0,108457.0,45474.0,271.0,9205.0,96400.0,690.0,14966.0,1756.0,45489.0,255.0,9206.0,55510.0,1196.0,56573.0,10.0,86.0,37.0,53375.0,110010.0,20.0,45418.0,117928.0,59.0,45418.0,117928.0,59.0,53375.0,110010.0,20.0,0.0,0.0,0.0,0.0,0.0,494.976862,563.021386,522.288325,530.445887,507.402483,483.20,575.2,525.8,510.80,520.0,869.6,850.4,810.2,996.1,1000.0,76.851781,84.013442,75.044063,102.763698,197.063769
1,2018,Centro-Oeste,MS,64081,413.0,8555.0,10894.0,6765.0,4952.0,3792.0,3044.0,2664.0,2293.0,2010.0,7008.0,4601.0,3288.0,1891.0,1024.0,529.0,225.0,92.0,34.0,7.0,37293.0,26788.0,1099.0,25296.0,5371.0,29228.0,1433.0,1654.0,17.0,61618.0,2115.0,136.0,195.0,40704.0,21039.0,1044.0,1294.0,46612.0,14726.0,55.0,2688.0,35460.0,231.0,6242.0,761.0,14726.0,55.0,2688.0,17230.0,1000.0,18210.0,4.0,1.0,15.0,22831.0,41231.0,11.0,19566.0,44462.0,45.0,19566.0,44462.0,45.0,22831.0,41231.0,11.0,0.0,0.0,0.0,0.0,0.0,493.407688,563.197069,520.302359,522.861165,486.249831,483.90,575.7,523.3,503.10,480.0,800.3,793.0,743.5,977.9,980.0,72.388278,80.548663,72.586982,99.530036,185.201462
2,2018,Centro-Oeste,MT,86051,969.0,11631.0,16124.0,9581.0,6569.0,5165.0,4090.0,3678.0,2962.0,2657.0,8802.0,5527.0,3819.0,2230.0,1205.0,606.0,270.0,124.0,29.0,13.0,50316.0,35735.0,1599.0,23450.0,12499.0,45723.0,2135.0,645.0,25.0,83071.0,2593.0,162.0,200.0,56441.0,27035.0,865.0,1710.0,64542.0,18388.0,49.0,3072.0,46967.0,314.0,6220.0,1907.0,18387.0,48.0,3073.0,21702.0,1713.0,23369.0,9.0,0.0,37.0,31322.0,54713.0,12.0,27002.0,59017.0,28.0,27002.0,59017.0,28.0,31322.0,54713.0,12.0,0.0,0.0,0.0,0.0,0.0,483.841582,554.556560,511.622926,516.115519,467.385330,473.40,564.4,513.5,498.00,460.0,815.7,850.4,759.9,986.4,1000.0,69.112267,81.518291,71.533420,95.616808,189.066749
3,2018,DF,DF,99503,567.0,11331.0,17350.0,12417.0,9060.0,6717.0,5108.0,4370.0,3472.0,2989.0,9670.0,6119.0,4298.0,2795.0,1724.0,939.0,360.0,155.0,45.0,17.0,58389.0,41114.0,2852.0,28758.0,14819.0,49614.0,2999.0,461.0,46.0,96550.0,2605.0,94.0,208.0,66524.0,30199.0,1129.0,1651.0,73810.0,18251.0,4.0,7438.0,50545.0,307.0,10696.0,677.0,18249.0,4.0,7440.0,25889.0,481.0,26328.0,19.0,1.0,22.0,30683.0,68803.0,6.

In [ ]:
df_result_uf.to_sql(name      = 'tb_enem_uf', 
                    con       = conn, 
                    if_exists = 'append', 
                    index     = False
                    
                   )

In [ ]:
df_result_municipality.to_sql(name      = 'tb_enem_municipality', 
                    con       = conn, 
                    if_exists = 'append', 
                    index     = False
                    
                   )

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

import pandas as pd
import json
from urllib.request import urlopen

# Importando os códigos das cidades
with urlopen('https://raw.githubusercontent.com/tbrugz/geodata-br/master/geojson/geojs-35-mun.json') as response:
    geo_json_sp = json.load( response )

# Gráfico
Figura = px.choropleth_mapbox(
  Base_Dados,
  geojson = geo_json_sp,
  locations='codigo_ibge',
  featureidkey = 'properties.id',
  color = 'Media_Nota_Geral',
  hover_data = ['NO_MUNICIPIO_PROVA'],
  color_continuous_scale='Viridis',
  mapbox_style = 'carto-positron',
  center = {'lat':-23.663889, 'lon': -46.537778},
  zoom = 6,
  opacity = 0.65, 
)
# Ajuste do Geo
Figura.update_geos(
    fitbounds='locations', 
    visible=False,
)
# Ajuste Layout
Figura.update_layout(
    autosize=True,
    margin=go.layout.Margin(l=0, r=0, t=0, b=0),
    showlegend=False,
    coloraxis_showscale=False
)